In [3]:
!cd /content

The system cannot find the path specified.


In [4]:
#from google.colab import files

In [6]:
files.upload()

NameError: name 'files' is not defined

In [ ]:
!mkdir ~/.kaggle

In [ ]:
!mv ./kaggle.json ~/.kaggle

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!pip install kaggle

In [ ]:
!kaggle datasets download -d kazanova/sentiment140

 80% 65.0M/80.9M [00:00<00:00, 120MB/s]
100% 80.9M/80.9M [00:00<00:00, 122MB/s]


In [ ]:
!unzip /content/sentiment140.zip

Archive:  /content/sentiment140.zip
  inflating: training.1600000.processed.noemoticon.csv  


In [ ]:
!pip install contractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 13.1 MB/s eta 0:00:00


In [ ]:
from concurrent.futures import ThreadPoolExecutor
from multiprocessing import cpu_count
import contractions
import re
from nltk import word_tokenize
import nltk
from nltk.corpus import stopwords
from spacy.lang.en.stop_words import STOP_WORDS
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn

In [ ]:
import pandas as pd
from keras.layers import TextVectorization
import numpy as np
from keras.layers import Input, Embedding, LSTM, GRU, Dense
from keras.models import Sequential
import pickle
import locale
import tensorflow as tf
import os

In [ ]:
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
nltk.download("averaged_perceptron_tagger")

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
nltk.download("wordnet")

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
cpu_count()

2

In [ ]:
data = pd.read_csv("/content/training.1600000.processed.noemoticon.csv",
                   encoding="latin-1",header=None)

In [ ]:
data.shape

(1600000, 6)

In [ ]:
data.head()

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
data.rename(columns={0:"target",1:"ids",2:"date",3:"flag",4:"user",5:"text"},
            inplace=True)

In [ ]:
data.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
data["text"].iloc[0]

"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D"

In [ ]:
reduced_data = data.drop(labels=data.columns[1:5],axis=1)

In [ ]:
reduced_data.head()

,target,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [ ]:
reduced_data["text"].head()

0    @switchfoot http://twitpic.com/2y1zl - Awww, t...
1    is upset that he can't update his Facebook by ...
2    @Kenichan I dived many times for the ball. Man...
3      my whole body feels itchy and like its on fire 
4    @nationwideclass no, it's not behaving at all....
Name: text, dtype: object

In [ ]:
def normalize_tweet(tweet_text):

  return tweet_text.lower()

In [ ]:
with ThreadPoolExecutor(max_workers=4) as pool:

  reduced_data["text"] = list(pool.map(normalize_tweet,list(reduced_data["text"])))

In [ ]:
reduced_data["text"].head()

0    @switchfoot http://twitpic.com/2y1zl - awww, t...
1    is upset that he can't update his facebook by ...
2    @kenichan i dived many times for the ball. man...
3      my whole body feels itchy and like its on fire 
4    @nationwideclass no, it's not behaving at all....
Name: text, dtype: object

In [ ]:
def expand_contractions(tweet_text):

  return contractions.fix(tweet_text)

In [ ]:
with ThreadPoolExecutor(max_workers=4) as pool:

  reduced_data["text"] = list(pool.map(expand_contractions,list(reduced_data["text"])))

In [ ]:
reduced_data["text"].head()

0    @switchfoot http://twitpic.com/2y1zl - awww, t...
1    is upset that he cannot update his facebook by...
2    @kenichan i dived many times for the ball. man...
3      my whole body feels itchy and like its on fire 
4    @nationwideclass no, it is not behaving at all...
Name: text, dtype: object

In [ ]:
regex_pattern = r'@[a-zA-z0-9 ]+|#[a-zA-Z0-9 ]+|\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*|\W+|\d+|<("[^"]*"|\'[^\']*\'|[^\'">])*>|_+|[^\u0000-\u007f]+'

In [ ]:
def remove_noisy_tokens(tweet_text):

  return re.sub(pattern=regex_pattern,repl=" ",string=tweet_text)

In [ ]:
with ThreadPoolExecutor(max_workers=4) as pool:

  reduced_data["text"] = list(pool.map(remove_noisy_tokens,list(reduced_data["text"])))

In [ ]:
reduced_data["text"].head()

0      twitpic com  y zl awww that is a bummer you ...
1    is upset that he cannot update his facebook by...
2          managed to save   the rest go out of bounds
3      my whole body feels itchy and like its on fire 
4      it is not behaving at all i am mad why am i ...
Name: text, dtype: object

In [ ]:
def remove_remaining_noisy_tokens(tweet_text):

  return re.sub(r'\b\w\b|[^\u0000-\u007f]+|_+|\W+',repl=" ",string=tweet_text)

In [ ]:
with ThreadPoolExecutor(max_workers=4) as pool:

  reduced_data["text"] = list(pool.map(remove_remaining_noisy_tokens,list(reduced_data["text"])))

In [ ]:
reduced_data["text"].head()

0     twitpic com   zl awww that is   bummer you sh...
1    is upset that he cannot update his facebook by...
2            managed to save the rest go out of bounds
3      my whole body feels itchy and like its on fire 
4     it is not behaving at all   am mad why am   h...
Name: text, dtype: object

In [ ]:
def tokenize_tweet_text(tweet_text):

  return word_tokenize(tweet_text)

In [ ]:
with ThreadPoolExecutor(max_workers=4) as pool:

  reduced_data["text"] = list(pool.map(tokenize_tweet_text,list(reduced_data["text"])))

In [ ]:
reduced_data["text"].head()

0    [twitpic, com, zl, awww, that, is, bummer, you...
1    [is, upset, that, he, can, not, update, his, f...
2    [managed, to, save, the, rest, go, out, of, bo...
3    [my, whole, body, feels, itchy, and, like, its...
4    [it, is, not, behaving, at, all, am, mad, why,...
Name: text, dtype: object

In [ ]:
en_stop_words = list(set(stopwords.words('english')).union(set(STOP_WORDS)))

In [ ]:
def is_stopword(token):

    return not(token in en_stop_words)

In [ ]:
def remove_stopwords(tweet_text):

  return list(filter(is_stopword,tweet_text))

In [ ]:
with ThreadPoolExecutor(max_workers=4) as pool:

  reduced_data["text"] = list(pool.map(remove_stopwords,list(reduced_data["text"])))

In [ ]:
reduced_data["text"].head()

0    [twitpic, com, zl, awww, bummer, shoulda, got,...
1    [upset, update, facebook, texting, cry, result...
2                        [managed, save, rest, bounds]
3                     [body, feels, itchy, like, fire]
4                                      [behaving, mad]
Name: text, dtype: object

In [ ]:
def get_wnet_pos_tag(treebank_tag):
    if treebank_tag[1].startswith('J'):
        return (treebank_tag[0],wn.ADJ)
    elif treebank_tag[1].startswith('V'):
        return (treebank_tag[0],wn.VERB)
    elif treebank_tag[1].startswith('N'):
        return (treebank_tag[0],wn.NOUN)
    elif treebank_tag[1].startswith('R'):
        return (treebank_tag[0],wn.ADV)
    else:
        return (treebank_tag[0],wn.NOUN)

In [ ]:
def get_pos_tag(tweet_text):

    return list(map(get_wnet_pos_tag,pos_tag(tweet_text)))

In [ ]:
reduced_data["text"] = list(map(get_pos_tag,list(reduced_data["text"])))

In [ ]:
reduced_data["text"].head()

0    [(twitpic, n), (com, n), (zl, n), (awww, n), (...
1    [(upset, a), (update, a), (facebook, n), (text...
2    [(managed, v), (save, v), (rest, n), (bounds, n)]
3    [(body, n), (feels, n), (itchy, v), (like, n),...
4                            [(behaving, v), (mad, n)]
Name: text, dtype: object

In [ ]:
lemmatizer = WordNetLemmatizer()

In [ ]:
def lemmatize_token(token_pos_tuple):

    if token_pos_tuple == None:
        return ""
    else:
        return lemmatizer.lemmatize(word=token_pos_tuple[0],pos=token_pos_tuple[1])

In [ ]:
def lemmatize_text(tweet_text):

    if len(tweet_text) > 0:
        return list(map(lemmatize_token,tweet_text))
    else:
        return [""]

In [ ]:
reduced_data["text"] = list(map(lemmatize_text,list(reduced_data["text"])))

In [ ]:
reduced_data["text"].head()

0    [twitpic, com, zl, awww, bummer, shoulda, get,...
1    [upset, update, facebook, texting, cry, result...
2                          [manage, save, rest, bound]
3                      [body, feel, itchy, like, fire]
4                                        [behave, mad]
Name: text, dtype: object

In [ ]:
with open("/content/tokenized_data.pkl","wb") as file_handle:

  pickle.dump(reduced_data,file_handle)

In [ ]:
!mv /content/tokenized_data.pkl /content/drive/MyDrive

In [ ]:
!cp /content/drive/MyDrive/tokenized_data.pkl /content

In [ ]:
with open("/content/tokenized_data.pkl","rb") as file_handle:

  reduced_data = pickle.load(file_handle)

In [ ]:
reduced_data.head()

,target,text
0,0,"[twitpic, com, zl, awww, bummer, shoulda, get,..."
1,0,"[upset, update, facebook, texting, cry, result..."
2,0,"[manage, save, rest, bound]"
3,0,"[body, feel, itchy, like, fire]"
4,0,"[behave, mad]"


In [ ]:
max_tokens = 30000
max_sequence_len = max(list(reduced_data["text"].apply(lambda x: len(x))))

In [ ]:
reduced_rows_idx = np.argwhere(np.array(reduced_data["text"].apply(lambda x: len(x))) >= 10)

In [ ]:
reduced_data = reduced_data.iloc[reduced_rows_idx.reshape(reduced_rows_idx.shape[0],)]

In [ ]:
reduced_data["text"] = list(map(lambda x: " ".join(x), list(reduced_data["text"])))

In [ ]:
reduced_data.head()

,target,text
0,0,twitpic com zl awww bummer shoulda get david c...
16,0,hollis death scene hurt severely watch film wr...
35,0,ok sick spend hour sit shower sick stand hold ...
39,0,bed class work gym class day go fly miss girlf...
44,0,fall asleep hear tracy girl body find sad hear...


In [ ]:
filtered_reduced_data = reduced_data[reduced_data["text"] != ""]

In [ ]:
filtered_reduced_data.shape

(214193, 2)

In [ ]:
vectorize_layer = TextVectorization(max_tokens=max_tokens,output_sequence_length=max_sequence_len)

In [ ]:
vectorize_layer.adapt(data=np.array(filtered_reduced_data["text"]))

In [ ]:
with open("/content/token_integer_mapping.pkl","wb") as file_handle:

  pickle.dump({"config":vectorize_layer.get_config(),
               "weights":vectorize_layer.get_weights()},file_handle)

In [ ]:
!mv /content/token_integer_mapping.pkl /content/drive/MyDrive

In [ ]:
!cp /content/drive/MyDrive/token_integer_mapping.pkl /content

In [ ]:
with open("/content/token_integer_mapping.pkl","rb") as file_handle:

  vectorize_layer_attributes = pickle.load(file_handle)

In [ ]:
loaded_vectorization_layer = TextVectorization.from_config(vectorize_layer_attributes["config"])

In [ ]:
loaded_vectorization_layer.set_weights(vectorize_layer_attributes["weights"])

In [ ]:
def vectorize_tweet(raw_tweet):

  return loaded_vectorization_layer(raw_tweet).numpy()

In [ ]:
vectorized_tweets = list(filtered_reduced_data["text"].apply(vectorize_tweet))

In [ ]:
vectorized_train_tweets = vectorized_tweets[0:200000]
vectorized_cv_tweets = vectorized_tweets[200000:]

In [ ]:
np.savez_compressed("./vectorized_train_tweets.npz",*vectorized_train_tweets)
np.savez_compressed("./vectorized_cv_tweets.npz",*vectorized_cv_tweets)

In [ ]:
!mv /content/vectorized_train_tweets.npz /content/drive/MyDrive
!mv /content/vectorized_cv_tweets.npz /content/drive/MyDrive

In [ ]:
!cp /content/drive/MyDrive/vectorized_train_tweets.npz /content
!cp /content/drive/MyDrive/vectorized_cv_tweets.npz /content

In [ ]:
filtered_reduced_data["target"] = filtered_reduced_data["target"].apply(lambda x: str(x))

In [ ]:
tweet_labels = list(filtered_reduced_data["target"].replace(to_replace=filtered_reduced_data["target"].unique(),
                                        value=list(range(len(filtered_reduced_data["target"].unique())))))

In [ ]:
train_labels = tweet_labels[0:200000]
cv_labels = tweet_labels[200000:]

In [ ]:
train_mb_size = 1000
num_epochs = 10
train_size = 200000

In [ ]:
loaded_vectorized_train_tweets = np.load("./vectorized_train_tweets.npz")

In [ ]:
def train_datagen():

  for _ in range(num_epochs):

    for i in range(train_size//train_mb_size):

      tweets_mb_list = [loaded_vectorized_train_tweets["arr_"+str(arr_idx)] for arr_idx in range(i*train_mb_size,(i+1)*train_mb_size)]
      tweets_labels_mb_list = [np.array(tweet_label) for tweet_label in train_labels[i*train_mb_size:(i+1)*train_mb_size]]

      yield np.array(tweets_mb_list), np.expand_dims(np.array(tweets_labels_mb_list),-1)

In [ ]:
for tweets_mb,tweets_labels in train_datagen():

  print(tweets_mb.shape)
  print(tweets_labels.shape)

  break

(1000, 50)
(1000, 1)


In [ ]:
cv_mb_size = 1577
cv_size = 14193

In [ ]:
loaded_vectorized_cv_tweets = np.load("./vectorized_cv_tweets.npz")

In [ ]:
def cv_datagen():

  for _ in range(num_epochs):

    for i in range(cv_size//cv_mb_size):

      tweets_mb_list = [loaded_vectorized_cv_tweets["arr_"+str(arr_idx)] for arr_idx in range(i*cv_mb_size,(i+1)*cv_mb_size)]
      tweets_labels_mb_list = [np.array(tweet_label) for tweet_label in cv_labels[i*cv_mb_size:(i+1)*cv_mb_size]]

      yield np.array(tweets_mb_list), np.expand_dims(np.array(tweets_labels_mb_list),-1)

In [ ]:
for tweets_mb,tweets_labels in cv_datagen():

  print(tweets_mb.shape)
  print(tweets_labels.shape)

  break

(1577, 50)
(1577, 1)


In [ ]:
!wget https://nlp.stanford.edu/data/glove.6B.zip

--2023-04-07 11:39:43--  https://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-04-07 11:39:44--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  5.02MB/s    in 2m 39s  

2023-04-07 11:42:23 (5.17 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]



In [ ]:
!unzip /content/glove.6B.zip

Archive:  /content/glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [ ]:
max_vocabulary_size = len(loaded_vectorization_layer.get_vocabulary())
embedding_output_dim = 50

In [ ]:
def create_bin_class_rnn():

  rnn_model = Sequential()

  rnn_model.add(Input(shape=(None,),dtype="int64"))
  rnn_model.add(Embedding(input_dim=max_vocabulary_size,output_dim=embedding_output_dim,input_length=max_sequence_len))
  rnn_model.add(LSTM(units=50))
  rnn_model.add(Dense(units=1,activation="sigmoid"))

  return rnn_model

In [ ]:
rnn_model = create_bin_class_rnn()

In [ ]:
rnn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 50)          1500000   
                                                                 
 lstm (LSTM)                 (None, 50)                20200     
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
Total params: 1,520,251
Trainable params: 1,520,251
Non-trainable params: 0
_________________________________________________________________


In [ ]:
rnn_model.compile(loss="binary_crossentropy",metrics=["accuracy",
                                                      tf.keras.metrics.Precision(),
                                                      tf.keras.metrics.Recall()])

In [ ]:
training_data_gen = train_datagen()
cv_data_gen = cv_datagen()

In [ ]:
rnn_model.fit(training_data_gen,
              epochs=num_epochs,
              validation_data=cv_data_gen,
              steps_per_epoch=200,validation_steps=9)

Epoch 1/10
200/200 [==============================] - 1439s 7s/step - loss: 0.2131 - accuracy: 0.9367 - precision: 0.9870 - recall: 0.8669 - val_loss: 0.0054 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 2/10
200/200 [==============================] - 1399s 7s/step - loss: 0.2283 - accuracy: 0.9267 - precision: 0.9011 - recall: 0.9353 - val_loss: 0.0021 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 3/10
200/200 [==============================] - 1389s 7s/step - loss: 0.1740 - accuracy: 0.9553 - precision: 0.9339 - recall: 0.9664 - val_loss: 0.0011 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 4/10
  7/200 [>.............................] - ETA: 15:08 - loss: 2.1023 - accuracy: 0.5714 - precision: 0.0000e+00 - recall: 0.0000e+00